In [1]:
import pyproj
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Some of the _x / _y columns are swapped. We can check whether the
# coordinates are actually in Switzerland rather than in the Atlantic
# Ocean using these constants:
# https://de.wikipedia.org/wiki/Geographische_Extrempunkte_der_Schweiz

LAT_MIN_WGS84 = 45.81796
LAT_MAX_WGS84 = 47.80845
LON_MIN_WGS84 = 5.95590
LON_MAX_WGS84 = 10.49219


In [2]:
%matplotlib widget

pv_data = pd.read_csv("../../data/pv_data_merged.csv")


In [3]:
# Check whether the coordinates are in Switzerland or not (and hence
# need to be swapped).

LV95 = "epsg:2056"
WGS84 = "epsg:4326"

transformer = pyproj.Transformer.from_crs(LV95, WGS84)


def swapped(row) -> bool:
    lat, lon = transformer.transform(row._x, row._y)
    if (
        lat < LAT_MIN_WGS84
        or lon > LON_MAX_WGS84
        or lat > LAT_MAX_WGS84
        or lon < LON_MIN_WGS84
    ):
        return True
    return False


pv_data["swapped"] = pv_data.apply(swapped, axis=1)


In [4]:
# Swap the coordinates if necessary. (This whole process is done in two
# steps because we want to).
pv_data.loc[pv_data["swapped"], ['_x', '_y']] = pv_data.loc[pv_data["swapped"], ['_y', '_x']].values

In [5]:
# Convert the coordinates from LV95 to WGS84.
pv_data["lat"], pv_data["lon"] = transformer.transform(pv_data["_x"], pv_data["_y"])

In [6]:
# Map the orientation to an actual Azimuth angle.

lut = {
    "Nord": 0.0,
    "Nordost": 45.0,
    "Ost": 90.0,
    "Südost": 135.0,
    "Süd": 180.0,
    "Südwest": 225.0,
    "West": 270.0,
    "Nordwest": 315.0,
    "Nachführsystem": 0.0, # This way it at least it contributes!
    "ohne Neigung montiert": 0.0,
    "Sonstige": 0.0,
}

pv_data["orientation_deg"] = pv_data["Ausrichtung"].map(lut)


In [9]:
# For some reason one value is inf after the conversion. We can just
# drop it.

import numpy as np
pv_data.replace([np.inf, -np.inf], np.nan, inplace=True)
pv_data.dropna(inplace=True, subset=["lat", "lon"], how="any")
pv_data.reset_index(inplace=True)

In [10]:
pv_data

,index,Unnamed: 0,xtf_id,Address,PostCode,Municipality,Canton,BeginningOfOperation,InitialPower,TotalPower,...,ID der Erw.,Anlagenkategorie,Ausrichtung,Neigungswinkel [°] Winkel gegen die Horizontale,Leistung pro Anlagenteil,Realisierte Leistung,swapped,lat,lon,orientation_deg
0,0,0,17313,Wiesengrundstrasse 2,8216,Oberhallau,SH,2011-10-19,115.10,115.10,...,NaN,Angebaute Anlage,Süd,25,14.88,14.88,False,47.702742,8.476378,180.0
1,1,1,11575,Rietacker,8526,Oberneunforn,TG,2008-03-13,29.90,29.90,...,NaN,Angebaute Anlage,Süd,12,29.90,29.90,False,47.603377,8.777378,180.0
2,2,2,10972,Haselstrasse 8,8335,Hittnau,ZH,2007-12-21,3.22,3.25,...,NaN,Integrierte Anlage,Süd,40,325.00,325.00,False,47.380154,8.836360,180.0
3,3,3,10986,Vers la Maison 11A,2715,Monible,BE,2008-05-29,3.24,6.55,...,17975.0,Integrierte Anlage,Süd,25,6.55,6.55,False,47.277081,7.201551,180.0
4,4,4,25142,Unteres Zihl 6,3127,Mühlethurnen,BE,2010-11-20,16.40,34.22,...,NaN,Integrierte Anlage,Süd,15,16.40,34.22,False,46.818662,7.505897,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44866,44867,45804,160104,Steinegg 12,9042,Speicher,AR,2012-07-09,6.54,6.54,...,NaN,NaN,Süd,30,6.54,6.54,False,47.407849,9.439641,180.0
44867,44868,45805,171193,Feldmoos,6162,Rengg,LU,2012-07-20,15.40,15.40,...,NaN,NaN,Ost,15,9.75,15.38,False,46.990084,8.088430,90.0
44868,44869,45806,171193,Feldmoos,6162,Rengg,LU,2012-07-20,15.40,15.40,...,NaN,NaN,West,90,5.63,15.38,False,46.990084,8.088430,270.0
44869,44870,45807,180465,Tägermoosstrasse 27,8700,Küsnacht,ZH,2020-12-22,260.22,260.22,...,NaN,NaN,Sonstige,10,260.22,260.22,False,47.323577,8.597088,0.0


In [11]:
# Rename columns to conform to the data model.

pv_data["installed_capacity_kw"] = pv_data["Leistung pro Anlagenteil"]

pv_data["address"] = pv_data["Address"]
pv_data["zipcode"] = pv_data["PostCode"]
pv_data["municipality"] = pv_data["Municipality"]
pv_data["canton"] = pv_data["Canton"]

pv_data["slope_deg"] = pv_data["Neigungswinkel [°] Winkel gegen die Horizontale"]

# These get populated later via the database.
pv_data["power_kw"] = float("nan")
pv_data["nearest_meter_id"] = float("nan")

pv_data["id"] = pv_data.index



In [12]:
data_clean = pv_data[
    [
        "id",
        "xtf_id",
        "lat",
        "lon",
        "address",
        "zipcode",
        "municipality",
        "canton",
        "installed_capacity_kw",
        "slope_deg",
        "orientation_deg",
        "nearest_meter_id",
        "power_kw",
    ]
]
data_clean

,id,xtf_id,lat,lon,address,zipcode,municipality,canton,installed_capacity_kw,slope_deg,orientation_deg,nearest_meter_id,power_kw
0,0,17313,47.702742,8.476378,Wiesengrundstrasse 2,8216,Oberhallau,SH,14.88,25,180.0,NaN,NaN
1,1,11575,47.603377,8.777378,Rietacker,8526,Oberneunforn,TG,29.90,12,180.0,NaN,NaN
2,2,10972,47.380154,8.836360,Haselstrasse 8,8335,Hittnau,ZH,325.00,40,180.0,NaN,NaN
3,3,10986,47.277081,7.201551,Vers la Maison 11A,2715,Monible,BE,6.55,25,180.0,NaN,NaN
4,4,25142,46.818662,7.505897,Unteres Zihl 6,3127,Mühlethurnen,BE,16.40,15,180.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44866,44866,160104,47.407849,9.439641,Steinegg 12,9042,Speicher,AR,6.54,30,180.0,NaN,NaN
44867,44867,171193,46.990084,8.088430,Feldmoos,6162,Rengg,LU,9.75,15,90.0,NaN,NaN
44868,44868,171193,46.990084,8.088430,Feldmoos,6162,Rengg,LU,5.63,90,270.0,NaN,NaN
44869,44869,180465,47.323577,8.597088,Tägermoosstrasse 27,8700,Küsnacht,ZH,260.22,10,0.0,NaN,NaN


In [13]:

data_clean.to_pickle("../../data/pv_plants.pkl")
